<a href="https://colab.research.google.com/github/Ashvin7/pl-xg-ml/blob/main/06_season_simulation_final.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Phase 6 — Season Simulation (Monte Carlo Forecasting)
This notebook simulates full Premier League seasons using match-level data (2017–2025) to produce:

Win/Draw/Loss (W/D/L) probabilities per matchup
Monte Carlo season simulations (e.g., 10,000 seasons)
Probability distributions for:
League position (1–20)
Title / Top-4 / Relegation
Expected points and uncertainty
Approach:
We fit a simple match-outcome model using team strength parameters (attack/defense + home advantage) from historical match results, then simulate seasons by sampling match outcomes using predicted probabilities.


In [22]:
!git clone https://github.com/Ashvin7/pl-xg-ml.git
%cd pl-xg-ml
!ls

Cloning into 'pl-xg-ml'...
remote: Enumerating objects: 52, done.
remote: Counting objects: 100% (52/52), done.
remote: Compressing objects: 100% (49/49), done.
remote: Total 52 (delta 20), reused 0 (delta 0), pack-reused 0 (from 0)
Receiving objects: 100% (52/52), 405.74 KiB | 9.66 MiB/s, done.
Resolving deltas: 100% (20/20), done.
/content/pl-xg-ml/pl-xg-ml
01_eda.ipynb		      05_match_results_ingestion.ipynb
01_phase1_data_prep.ipynb     data
02_feature_engineering.ipynb  notebooks
03_baseline_models.ipynb      Phase_0_Project_SetUp.ipynb
04_ensemble_models.ipynb      README.md


In [2]:
!pip -q install statsmodels joblib

import os
import numpy as np
import pandas as pd
import joblib

import statsmodels.api as sm
import statsmodels.formula.api as smf


In [3]:
MATCH_PATH = "data/processed/epl_match_results_2017_2025.csv"

OUT_MODEL = "models/phase5_poisson_decay_fixed_model.joblib"
OUT_SUMMARY = "data/processed/phase5_poisson_decay_fixed_title_top4_relegation_probs.csv"
OUT_POS_PROBS = "data/processed/phase5_poisson_decay_fixed_position_probabilities.csv"

os.makedirs("models", exist_ok=True)
os.makedirs("data/processed", exist_ok=True)

print("Match exists:", os.path.exists(MATCH_PATH))

Match exists: True


In [4]:
matches = pd.read_csv(MATCH_PATH)
print(matches.shape)
print(matches.columns.tolist())
matches.head()

(3010, 9)
['date', 'season', 'home_team', 'away_team', 'home_goals', 'away_goals', 'result', 'home_points', 'away_points']


,date,season,home_team,away_team,home_goals,away_goals,result,home_points,away_points
0,2017-08-11,2017/18,Arsenal,Leicester,4,3,H,3,0
1,2017-08-12,2017/18,Brighton,Man City,0,2,A,0,3
2,2017-08-12,2017/18,Chelsea,Burnley,2,3,A,0,3
3,2017-08-12,2017/18,Crystal Palace,Huddersfield,0,3,A,0,3
4,2017-08-12,2017/18,Everton,Stoke,1,0,H,3,0


In [5]:
df = matches.copy()

df["date"] = pd.to_datetime(df["date"], errors="coerce")
df["home_goals"] = pd.to_numeric(df["home_goals"], errors="coerce")
df["away_goals"] = pd.to_numeric(df["away_goals"], errors="coerce")

df = df.dropna(subset=["season","date","home_team","away_team","home_goals","away_goals"]).copy()
df["home_goals"] = df["home_goals"].astype(int)
df["away_goals"] = df["away_goals"].astype(int)

df.shape

(3010, 9)

In [6]:
HALF_LIFE_DAYS = 365  # try 180 if you want more recency emphasis

max_date = df["date"].max()
age_days = (max_date - df["date"]).dt.days.astype(float)

k = np.log(2) / HALF_LIFE_DAYS
df["weight"] = np.exp(-k * age_days)

print("Most recent weight:", df["weight"].max())
print("Oldest weight:", df["weight"].min())
df[["date","weight"]].describe()


Most recent weight: 1.0
Oldest weight: 0.004687432720139878


,date,weight
count,3010,3010.000000
mean,2021-06-25 21:16:51.827242496,0.193485
min,2017-08-11 00:00:00,0.004687
25%,2019-05-12 00:00:00,0.015774
50%,2021-05-19 00:00:00,0.064062
75%,2023-05-17 06:00:00,0.255399
max,2025-05-05 00:00:00,1.000000
std,NaN,0.254918


In [7]:
long_df = pd.concat([
    pd.DataFrame({
        "team": df["home_team"],
        "opponent": df["away_team"],
        "goals": df["home_goals"],
        "is_home": 1,
        "weight": df["weight"]
    }),
    pd.DataFrame({
        "team": df["away_team"],
        "opponent": df["home_team"],
        "goals": df["away_goals"],
        "is_home": 0,
        "weight": df["weight"]
    }),
], ignore_index=True)

long_df.shape, long_df.head()

((6020, 5),
              team      opponent  goals  is_home    weight
 0         Arsenal     Leicester      4        1  0.004687
 1        Brighton      Man City      0        1  0.004696
 2         Chelsea       Burnley      2        1  0.004696
 3  Crystal Palace  Huddersfield      0        1  0.004696
 4         Everton         Stoke      1        1  0.004696)

In [8]:
poisson_model = smf.glm(
    formula="goals ~ is_home + C(team) + C(opponent)",
    data=long_df,
    family=sm.families.Poisson(),
    freq_weights=long_df["weight"]
).fit()

print(poisson_model.summary().tables[0])


                 Generalized Linear Model Regression Results                  
Dep. Variable:                  goals   No. Observations:                 6020
Model:                            GLM   Df Residuals:                  1102.78
Model Family:                 Poisson   Df Model:                           61
Link Function:                    Log   Scale:                          1.0000
Method:                          IRLS   Log-Likelihood:                -1727.4
Date:                Fri, 30 Jan 2026   Deviance:                       1243.9
Time:                        09:37:53   Pearson chi2:                 1.08e+03
No. Iterations:                     5   Pseudo R-squ. (CS):            0.03368
Covariance Type:            nonrobust                                         


In [9]:
import os
os.makedirs(os.path.dirname(OUT_MODEL), exist_ok=True)
joblib.dump(
    {"model": poisson_model, "half_life_days": HALF_LIFE_DAYS, "max_date": str(max_date)},
    OUT_MODEL
)
OUT_MODEL

'models/phase5_poisson_decay_fixed_model.joblib'

In [10]:
LAST_SEASON = sorted(df["season"].unique())[-1]

teams_next = sorted(
    set(df[df["season"] == LAST_SEASON]["home_team"]).union(
        set(df[df["season"] == LAST_SEASON]["away_team"])
    )
)

print("Last season:", LAST_SEASON)
print("Teams:", len(teams_next))
teams_next[:10]


Last season: 2024/25
Teams: 20


['Arsenal',
 'Aston Villa',
 'Bournemouth',
 'Brentford',
 'Brighton',
 'Chelsea',
 'Crystal Palace',
 'Everton',
 'Fulham',
 'Ipswich']

In [11]:
def generate_double_round_robin(teams):
    teams = list(teams)
    fixtures = []
    for i in range(len(teams)):
        for j in range(i+1, len(teams)):
            fixtures.append((teams[i], teams[j]))  # i home
            fixtures.append((teams[j], teams[i]))  # j home
    return pd.DataFrame(fixtures, columns=["home_team","away_team"])

fixtures = generate_double_round_robin(teams_next)
fixtures.shape, fixtures.head()


((380, 2),
      home_team    away_team
 0      Arsenal  Aston Villa
 1  Aston Villa      Arsenal
 2      Arsenal  Bournemouth
 3  Bournemouth      Arsenal
 4      Arsenal    Brentford)

In [12]:
def predict_lambda(model, team, opponent, is_home):
    X = pd.DataFrame({"team": team, "opponent": opponent, "is_home": is_home})
    return model.predict(X)

fixtures = fixtures.copy()

fixtures["lambda_home"] = predict_lambda(
    poisson_model,
    team=fixtures["home_team"],
    opponent=fixtures["away_team"],
    is_home=1
)

fixtures["lambda_away"] = predict_lambda(
    poisson_model,
    team=fixtures["away_team"],
    opponent=fixtures["home_team"],
    is_home=0
)

fixtures[["lambda_home","lambda_away"]].describe()


,lambda_home,lambda_away
count,380.000000,380.000000
mean,1.602461,1.392281
std,0.529627,0.460161
min,0.578216,0.502376
25%,1.210440,1.051678
50%,1.511784,1.313497
75%,1.945189,1.690056
max,3.393325,2.948253


In [13]:
rng = np.random.default_rng(42)

def simulate_season_scorelines(fixtures_df, n_sims=10000, rng=None):
    if rng is None:
        rng = np.random.default_rng(0)

    teams = pd.unique(fixtures_df[["home_team","away_team"]].values.ravel())
    team_to_idx = {t:i for i,t in enumerate(teams)}
    n_teams = len(teams)

    pts = np.zeros((n_sims, n_teams), dtype=np.int32)
    gf  = np.zeros((n_sims, n_teams), dtype=np.int32)
    ga  = np.zeros((n_sims, n_teams), dtype=np.int32)

    home_idx = fixtures_df["home_team"].map(team_to_idx).to_numpy()
    away_idx = fixtures_df["away_team"].map(team_to_idx).to_numpy()
    lam_h = fixtures_df["lambda_home"].to_numpy()
    lam_a = fixtures_df["lambda_away"].to_numpy()

    n_matches = len(fixtures_df)

    for s in range(n_sims):
        hg = rng.poisson(lam_h)
        ag = rng.poisson(lam_a)

        for i in range(n_matches):
            h = home_idx[i]; a = away_idx[i]

            gf[s, h] += hg[i]; ga[s, h] += ag[i]
            gf[s, a] += ag[i]; ga[s, a] += hg[i]

            if hg[i] > ag[i]:
                pts[s, h] += 3
            elif hg[i] < ag[i]:
                pts[s, a] += 3
            else:
                pts[s, h] += 1
                pts[s, a] += 1

    gd = gf - ga
    return teams, pts, gf, ga, gd


In [14]:
N_SIMS = 10000

teams_sim, pts_sims, gf_sims, ga_sims, gd_sims = simulate_season_scorelines(
    fixtures, n_sims=N_SIMS, rng=rng
)

pts_sims.shape


(10000, 20)

In [15]:
def compute_positions_with_tiebreakers(pts, gd, gf):
    n_sims, n_teams = pts.shape
    pos = np.empty((n_sims, n_teams), dtype=np.int16)

    for s in range(n_sims):
        # Sort ascending by pts, gd, gf, then reverse to get best-first
        order = np.lexsort((gf[s], gd[s], pts[s]))[::-1]  # BEST teams first
        pos[s, order] = np.arange(1, n_teams + 1)         # 1 = champion

    return pos

In [16]:
pos = compute_positions_with_tiebreakers(pts_sims, gd_sims, gf_sims)

In [17]:
s = 0
ranked = pd.DataFrame({
    "team": teams_sim,
    "pts": pts_sims[s],
    "gd": gd_sims[s],
    "gf": gf_sims[s],
    "pos": pos[s],
}).sort_values(["pos"])

ranked.head(5), ranked.tail(5)


(         team  pts  gd   gf  pos
 12   Man City   82  61  103    1
 11  Liverpool   82  51   88    2
 0     Arsenal   77  41   74    3
 14  Newcastle   74  40   86    4
 5     Chelsea   64   3   60    5,
               team  pts  gd  gf  pos
 6   Crystal Palace   42 -11  38   16
 10       Leicester   36 -29  38   17
 18        West Ham   31 -32  47   18
 16     Southampton   24 -61  27   19
 9          Ipswich   21 -58  28   20)

In [18]:
summary = pd.DataFrame({
    "team": teams_sim,
    "title": (pos == 1).mean(axis=0),
    "top4": (pos <= 4).mean(axis=0),
    "relegation": (pos >= 18).mean(axis=0),
    "avg_finish": pos.mean(axis=0),
    "avg_points": pts_sims.mean(axis=0),
    "avg_gd": gd_sims.mean(axis=0),
    "avg_gf": gf_sims.mean(axis=0),
}).sort_values("title", ascending=False)

summary.head(10)


,team,title,top4,relegation,avg_finish,avg_points,avg_gd,avg_gf
12,Man City,0.3942,0.9615,0.0000,2.0828,79.6104,45.6450,84.1218
11,Liverpool,0.3175,0.9485,0.0000,2.2869,78.4078,43.7361,84.0281
0,Arsenal,0.2607,0.9329,0.0000,2.4654,77.4239,40.0334,75.7045
14,Newcastle,0.0141,0.3776,0.0002,5.8040,64.0558,18.8095,71.2429
5,Chelsea,0.0083,0.2943,0.0004,6.3459,62.4771,15.8136,64.9981
1,Aston Villa,0.0014,0.1116,0.0025,8.3572,57.4335,7.5643,61.4575
3,Brentford,0.0014,0.0832,0.0058,8.9881,55.9667,5.1347,61.5097
17,Tottenham,0.0013,0.1180,0.0024,8.3586,57.4033,7.4377,68.1944
15,Nott'm Forest,0.0002,0.0318,0.0172,10.9875,51.5240,-1.8145,53.6031
8,Fulham,0.0002,0.0319,0.0149,10.9358,51.6532,-1.5131,52.8146


In [19]:
n_teams = len(teams_sim)

pos_rows = []
for i, t in enumerate(teams_sim):
    counts = np.bincount(pos[:, i], minlength=n_teams+1)[1:] / N_SIMS
    row = {"team": t}
    for p in range(1, n_teams+1):
        row[f"p_pos_{p}"] = counts[p-1]
    pos_rows.append(row)

pos_probs = pd.DataFrame(pos_rows)
pos_probs.head()


,team,p_pos_1,p_pos_2,p_pos_3,p_pos_4,p_pos_5,p_pos_6,p_pos_7,p_pos_8,p_pos_9,...,p_pos_11,p_pos_12,p_pos_13,p_pos_14,p_pos_15,p_pos_16,p_pos_17,p_pos_18,p_pos_19,p_pos_20
0,Arsenal,0.2607,0.2991,0.2733,0.0998,0.0393,0.0141,0.0075,0.0039,0.0013,...,0.0003,0.0001,0.0000,0.0001,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000
1,Aston Villa,0.0014,0.0074,0.0247,0.0781,0.1112,0.1235,0.1155,0.1060,0.0926,...,0.0670,0.0567,0.0489,0.0350,0.0259,0.0158,0.0085,0.0019,0.0005,0.0001
2,Bournemouth,0.0001,0.0007,0.0057,0.0218,0.0411,0.0573,0.0721,0.0760,0.0870,...,0.0885,0.0903,0.0882,0.0905,0.0701,0.0614,0.0400,0.0153,0.0025,0.0004
3,Brentford,0.0014,0.0056,0.0183,0.0579,0.0905,0.1034,0.1084,0.1032,0.0989,...,0.0785,0.0677,0.0540,0.0444,0.0377,0.0250,0.0112,0.0051,0.0006,0.0001
4,Brighton,0.0002,0.0011,0.0052,0.0207,0.0395,0.0551,0.0630,0.0740,0.0871,...,0.0927,0.0965,0.0898,0.0865,0.0759,0.0575,0.0448,0.0147,0.0032,0.0003


In [20]:
summary.to_csv(OUT_SUMMARY, index=False)
pos_probs.to_csv(OUT_POS_PROBS, index=False)

OUT_SUMMARY, OUT_POS_PROBS


('data/processed/phase5_poisson_decay_fixed_title_top4_relegation_probs.csv',
 'data/processed/phase5_poisson_decay_fixed_position_probabilities.csv')

In [21]:
display(summary.head(10)[["team","title","top4","relegation","avg_points","avg_gd"]])
display(summary.tail(10)[["team","title","top4","relegation","avg_points","avg_gd"]])


,team,title,top4,relegation,avg_points,avg_gd
12,Man City,0.3942,0.9615,0.0000,79.6104,45.6450
11,Liverpool,0.3175,0.9485,0.0000,78.4078,43.7361
0,Arsenal,0.2607,0.9329,0.0000,77.4239,40.0334
14,Newcastle,0.0141,0.3776,0.0002,64.0558,18.8095
5,Chelsea,0.0083,0.2943,0.0004,62.4771,15.8136
1,Aston Villa,0.0014,0.1116,0.0025,57.4335,7.5643
3,Brentford,0.0014,0.0832,0.0058,55.9667,5.1347
17,Tottenham,0.0013,0.1180,0.0024,57.4033,7.4377
15,Nott'm Forest,0.0002,0.0318,0.0172,51.5240,-1.8145
8,Fulham,0.0002,0.0319,0.0149,51.6532,-1.5131


,team,title,top4,relegation,avg_points,avg_gd
6,Crystal Palace,0.0002,0.0161,0.0271,49.6216,-4.2553
4,Brighton,0.0002,0.0272,0.0182,51.3434,-2.4925
13,Man United,0.0002,0.0272,0.0164,51.5544,-1.5925
2,Bournemouth,0.0001,0.0283,0.0182,51.5670,-1.6249
7,Everton,0.0000,0.0043,0.0857,44.7668,-10.4290
9,Ipswich,0.0000,0.0000,0.8682,28.4564,-43.5518
10,Leicester,0.0000,0.0001,0.7391,31.8283,-35.5322
16,Southampton,0.0000,0.0000,0.9722,22.8724,-52.9788
18,West Ham,0.0000,0.0022,0.1157,43.4952,-15.1955
19,Wolves,0.0000,0.0033,0.0958,44.4596,-13.1942
